### Step 1: Initial Data Loading

In [ ]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Cardiovascular_Disease/GSE181339'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [ ]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# 1. Gene Expression Data Availability
is_gene_available = True  # Given the information provided, the study involves gene expression analysis.
# as it’s based on gene expression analysis in blood cells.

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability
# Checking keys for each variable
sample_characteristics = {
    0: ['gender: Man', 'gender: Woman'],
    1: ['group: NW', 'group: OW/OB', 'group: MONW'],
    2: ['age: 21', 'age: 23', 'age: 10', 'age: 17', 'age: 11', 'age: 1', 'age: 18', 'age: 12', 'age: 8', 'age: 14', 'age: 26', 'age: 4', 'age: 2', 'age: 3', 'age: 7', 'age: 13', 'age: 15', 'age: 9', 'age: 30', 'age: 19'],
    3: ['fasting time: 6hr', 'fasting time: 4hr'],
    4: ['timepoint: 0months', 'timepoint: 6months']
}

# Checking the key dictionary for availability of variables
trait_row = 1  # 'group' key is inferred as 'Cardiovascular_Disease'
age_row = 2  # 'age' key directly available as 'age'
gender_row = 0  # 'gender' key directly available as 'gender'

# 2.3 Data Type Conversion
# Defining conversion functions

def convert_trait(value):
    """Convert Cardiovascular Disease value to binary."""
    mapping = {'group: NW': 0, 'group: MONW': 1, 'group: OW/OB': 1}
    return mapping.get(value, None)

def convert_age(value):
    """Convert age to continuous."""
    try:
        return float(value.split(': ')[1])
    except ValueError:
        return None

def convert_gender(value):
    """Convert gender to binary (female: 0, male: 1)."""
    mapping = {'gender: Woman': 0, 'gender: Man': 1}
    return mapping.get(value, None)

# 3. Save Metadata
save_cohort_info('GSE181339', './preprocessed/Cardiovascular_Disease/cohort_info.json', is_gene_available, trait_row is not None)

# 4. Clinical Feature Extraction
if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Cardiovascular_Disease', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Cardiovascular_Disease/trait_data/GSE181339.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))


### Step 3: Gene Data Extraction

In [ ]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row ids for the following step.
print(gene_data.index[:20])
    

# Based on the provided row headers from STEP3, it appears that these are numerical values and not human gene symbols. 
# Human gene symbols typically consist of letters and possibly numbers (e.g., TP53, BRCA1).


### Step 4: Gene Identifier Review

In [ ]:
requires_gene_mapping = True


### Step 5: Gene Annotation (Conditional)

In [ ]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the soft file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


### Step 6: Gene Identifier Mapping

In [ ]:
# 1. Determine the identifier key and gene symbol key from the gene annotation preview
identifier_key = 'SPOT_ID'
gene_symbol_key = 'GENE_SYMBOL'

# 2. Get the dataframe storing the mapping between probe IDs and genes
gene_mapping = get_gene_mapping(gene_annotation, identifier_key, gene_symbol_key)

# Ensure consistency in column naming for the mapping process (set the correct column name for ID)
gene_mapping = gene_mapping.rename(columns={identifier_key: 'ID', gene_symbol_key: 'Gene'})

# 3. Apply the mapping with the apply_gene_mapping function
gene_data = apply_gene_mapping(gene_data, gene_mapping)


### Step 7: Data Normalization and Merging

In [ ]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
gene_csv_path = './preprocessed/Cardiovascular_Disease/gene_data/GSE181339.csv'
normalized_gene_data.to_csv(gene_csv_path)

# 2. Merge the clinical and genetic data with the 'geo_merge_clinical_genetic_data' function from the library.
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, normalized_gene_data)

# 3. Determine whether the trait and some demographic attributes in the data are severely biased, and remove biased attributes.
trait_biased, unbiased_merged_data = judge_and_remove_biased_features(merged_data, 'Cardiovascular_Disease')

# 4. Save the cohort information.
save_cohort_info('GSE181339', './preprocessed/Cardiovascular_Disease/cohort_info.json', True, True, trait_biased, merged_data)

if not trait_biased:
    # 5. If the trait is not severely biased, save the merged data to a csv file.
    csv_path = './preprocessed/Cardiovascular_Disease/GSE181339.csv'
    unbiased_merged_data.to_csv(csv_path)
